In [1]:
import json

from OneM2M.ACME import ACME
from OneM2M.OM2M import OM2M
from OneM2M.Mobius import Mobius

In [2]:
with open('nodes.json') as f:
    nodes = json.load(f)

In [3]:
om2m = OM2M('admin', 'admin', 'http://10.3.1.117:8200/~/in-cse/in-name')

for node in nodes:
    print ("Creating node: " + node)
    r = om2m.create_AE(node)
    if r[0] != 200 and r[0] != 409 and r[0] != 201:
        print ("Error creating node: " + node)
        print (r)
        continue        
    for sensor in nodes[node]:
        print ("Creating sensor: " + sensor, end='\r')
        r = om2m.create_container(sensor, node)
        if r != 200 and r != 409 and r != 201:
            print ("Error creating sensor: " + sensor)
            print (r)
            exit(1)
        print ("Created sensor: " + sensor + ", response: " + str(r))

Creating node: AE-AQ
Created sensor: AQ-MG00-00, response: 201
Created sensor: AQ-BN00-00, response: 201
Created sensor: AQ-KN00-00, response: 201
Created sensor: AQ-SN00-00, response: 201
Created sensor: AQ-VN90-00, response: 201
Created sensor: AQ-KH00-00, response: 201
Created sensor: AQ-AN00-00, response: 201
Created sensor: AQ-PH03-00, response: 201
Created sensor: AQ-PL00-00, response: 201
Created sensor: AQ-FG00-00, response: 201
Created sensor: AQ-AD95-00, response: 201
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Creating node: AE-CM
Created sensor: CM-MG00-00, respon

In [5]:
mobius = Mobius('12345', 'http://10.3.1.117:8001/Mobius')
# mobius.create_AE('test', 'S123', ['Mobius'], True)
# mobius.create_container('testnode', 'test', 'S123', ['Mobius'], 120)

for node in nodes:
    print ("Creating node: " + node)
    origin = 'SOrigin' + node
    r = mobius.create_AE(node, origin)
    if r[0] != 200 and r[0] != 409 and r[0] != 201:
        print ("Error creating node: " + node)
        print (r)
        continue        
    for sensor in nodes[node]:
        print ("Creating sensor: " + sensor, end='\r')
        r = mobius.create_container(sensor, node, origin)
        if r != 200 and r != 409 and r != 201:
            print ("Error creating sensor: " + sensor)
            print (r)
            exit(1)
        print ("Created sensor: " + sensor + ", response: " + str(r))

Creating node: AE-AQ
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Creating node: AE-CM
Created sensor: CM-MG00-00, respon

In [4]:
acme = ACME('http://10.3.1.117:8002', 'ri.json', '3')
for node in nodes:
    print ("Creating node: " + node)
    r = acme.create_AE(node, '123')
    if r[0] != 200 and r[0] != 409 and r[0] != 201 and r[0] != 403:
        print ("Error creating node: " + node)
        print (r)
        continue        
    for sensor in nodes[node]:
        print ("Creating sensor: " + sensor, end='\r')
        r = acme.create_container(sensor, node, '123')
        if r != 200 and r != 409 and r != 201 and r != 403:
            print ("Error creating sensor: " + sensor)
            print (r)
            exit(1)
        print ("Created sensor: " + sensor + ", response: " + str(r))

Creating node: AE-AQ
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Creating node: AE-CM
Created sensor: CM-MG00-00, respon